In [108]:
import re
import pandas as pd

In [109]:
def parser(file):
    
    #HTML to Dataframe and Lat/Long Conversion
    data1 = pd.read_html(file, header=0)[2]
    
    data1['ATS'] = data1['Location  LE/LSD-SEC-TWP-RGEWM'].str.replace(r'[^0-9]+', '')
    data1['ATS'] = data1['ATS'].str[-8:]
    data1['ATS'] = data1['ATS'].astype('int64')

    #Extract Date
    lines = []
    with open(file, 'r') as file:
        for line in file:
            lines.append(line.rstrip('\n'))

    Date = (re.split('[\s]{4,}', lines[21].strip()))
    Date = Date[0]
    Date = Date.replace('Date:', '')
    Date = Date.replace('</b></td>', '')
    data1['DATE'] = Date

    convcomp = pd.read_csv('ConvComp.csv')
    convcomp['ConvDF'] = convcomp['ConvDF'].astype('int64')
    dfm = pd.merge(data1, convcomp, how='left', left_on='ATS', right_on='ConvDF')

    return dfm

In [110]:
import os

from pathlib import Path

directory_in_str = "ST97/"

directory = os.fsencode(directory_in_str) 

columns = ['Licensee Name', 'Application Number', 'Application Purpose',
       'Category Type Description', 'Location  LE/LSD-SEC-TWP-RGEWM',
       'Licence  Number', 'ATS', 'ConvDF', 'Lat', 'Long', 'DATE']

df = pd.DataFrame(columns=columns)

ext = ['.html','.HTML']

for file in os.listdir(directory): 
    filename = os.fsdecode(file) 
    if filename.endswith(tuple(ext)): 
        data = parser(os.path.join(directory_in_str, filename))
        df = df.append(data, ignore_index=True, sort=False)
        continue 
    else: 
        continue

In [111]:
df

,Licensee Name,Application Number,Application Purpose,Category Type Description,Location LE/LSD-SEC-TWP-RGEWM,Licence Number,ATS,ConvDF,Lat,Long,DATE
0,ORLEN Upstream Canada Ltd.,1928693,Licence Amendment,Gas processing plant < 0.01 mol/kmol H2S in in...,00/04-13-039-09W5,43942,13039095,13039095,52.355925,-115.152891,22 May 2020
1,Baccalieu Energy Inc.,1927852,Licence Amendment,Oil battery - multiwell < 0.01 mol/kmol H2S in...,03/02-10-048-12W5,50427,10048125,10048125,53.126924,-115.659509,20 March 2020
2,Chevron Canada Limited,1928532,Licence Amendment,Gas battery - multiwell < 1 t/d sulphur inlet,00/10-33-060-18W5,48902,33060185,33060185,54.232255,-116.626586,05 May 2020
3,Advantage Oil & Gas Ltd.,1926888,New Licence,Oil battery - multiwell >= 1 t/d sulphur inlet,00/04-21-076-09W6,51848,21076096,21076096,55.599525,-119.319147,04 February 2020
4,Altura Energy Inc.,1927038,Licence Amendment,Oil satellite - multiwell < 0.01 mol/kmol H2S ...,00/11-27-048-26W4,50823,27048264,27048264,53.170568,-113.707966,04 February 2020
...,...,...,...,...,...,...,...,...,...,...,...
280,Canadian Natural Resources Limited,1927147,New Licence,Bitumen battery - multiwell < 0.01 mol/kmol H2...,00/03-18-062-07W4,51871,18062074,18062074,54.362557,-111.030288,11 February 2020
281,MEG Energy Corp.,1926826,Licence Amendment,Bitumen battery - multiwell >= 1 t/d sulphur i...,00/02-16-077-05W4,37114,16077054,16077054,55.671811,-110.704494,11 February 2020
282,Rolling Hills Energy Ltd.,1927134,New Licence,Oil satellite - multiwell < 0.01 mol/kmol H2S ...,02/04-27-034-03W4,51870,27034034,27034034,51.947284,-110.335787,11 February 2020
283,TAQA North Ltd.,1927158,New Licence,Gas battery - multiwell < 0.01 mol/kmol H2S in...,00/04-03-044-11W5,51873,3044115,3044115,52.763447,-115.500689,11 February 2020


In [112]:
df.to_csv(r'ST97.csv', index = False)